In [1]:
from requests_html import HTMLSession
from requests_html import HTML
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
import time
import csv
import re

In [2]:
def ThuThapData(link,maxNumPage,fileName):
    f = open(fileName, "a",encoding="utf-8")
    f.write("Loại\tTên\tCác loại phòng\tGiá\tVị trí\tSố đánh giá\n")
    browser = webdriver.Chrome(executable_path='chromedriver.exe')
    for page in range(1,maxNumPage+1):
        url = f"{link}{page}"
        browser.get(url)
        time.sleep(3)
        r = HTML(html=browser.page_source)
        posts=r.find(".promo.is-relative")
        for stt,post in enumerate(posts):
            temp=post.text.split("\n")
            # Bỏ đi dòng khuyến mãi
            if temp[0][0]=='-':
                temp.remove(temp[0])
            # Bỏ đi trạng thái phòng
            if 'Đặt phòng ngay' in temp[1]:
                temp.remove(temp[1])
            for i in temp:
                f.write(i)
                f.write("\t")
    f.write(link)
    f.write("\t")
    f.write("\n")
    f.close()

In [3]:
#ThuThapData("https://www.luxstay.com/vi/vietnam/lam-dong/da-lat?page=",71,"dalat.csv")
#ThuThapData("https://www.luxstay.com/vi/vietnam/ha-noi?page=",93,"hanoi.csv")
#ThuThapData("https://www.luxstay.com/vi/vietnam/ho-chi-minh?page=",134,"hochiminh.csv")
#ThuThapData("https://www.luxstay.com/vi/vietnam/ba-ria-vung-tau/vung-tau?page=",31,"vungtau.csv")
#ThuThapData("https://www.luxstay.com/vi/vietnam/khanh-hoa/nha-trang?page=",34,"nhatrang.csv")
#ThuThapData("https://www.luxstay.com/vi/vietnam/quang-ninh?page=",15,"quangninh.csv")


In [4]:
def crawDataOnePage(browser,url,time_sleep,file_name):
    browser.get(url)
    browser.execute_script("window.scrollTo(0, window.scrollY + 1200)")
    time.sleep(time_sleep)
    # Tắt quảng cáo bằng ESC nếu có
    browser.find_element_by_tag_name('html').send_keys(Keys.ESCAPE)
    r = HTML(html=browser.page_source)

    f = open(file_name, "a+",encoding="utf-8")
    f.write(url)
    f.write("\t")
    # Lấy tên nhà
    try:
        f.write(r.find(".col-xs-12.col-sm-10")[0].text)
    except:
        pass
    f.write("\t")
    # Lấy loại nhà
    try:
        f.write(r.find(".mt--12")[0].text.split(" · ")[0])
    except:
        pass
    f.write("\t")
    # Lấy Diện tích
    try:
        f.write(r.find(".mt--12")[0].text.split(" · ")[1])
    except:
        pass  
    f.write("\t")
    # Lấy các lại phòng
    try:
        f.write(r.find(".mb--0")[2].text)
    except:
        pass  
    f.write("\t")
    # Lấy giá phòng
    try:
        f.write(r.find(".fadeIn")[1].text)
    except:
        pass 
    f.write("\t")
    # Lấy ví trí
    try:
        f.write(r.find(".ml--6.bold")[0].text)
    except:
        pass 
    f.write("\t")
    #lấy tiện ích
    for i in range(10):
        try:
            x=browser.find_element_by_link_text("Xem thêm")
            x.click()
            reponse = HTML(html=browser.page_source)
            break
        except:
            pass
    try:
        f.write(reponse.find(".container.container--sm.pb--48")[0].text.replace("\n",",")) 
    except:
        pass
    f.write("\n")
    f.close()

In [5]:
def crawDataMulPage(main_link,time_sleep,file_name,startNumPage,endNumPage):
    pattern=r"^/vi/rooms/\d+"
    
    fi = open(file_name, "w",encoding="utf-8")
    fi.write("Link\tTên\tLoại nhà\tDiện tích\tCác loại phòng\tGiá\tVị trí\tTiện ích\n")
    fi.close()
    browser = webdriver.Chrome(executable_path='chromedriver.exe')
    for page in range(startNumPage,endNumPage+1):
        main_url = f"{main_link}{page}"
        #print(main_url)
        browser.get(main_url)
        r = HTML(html=browser.page_source)
        sub_links=[]
        # lọc link cần
        for link in r.links:
            if re.match(pattern,link)!=None:
                sub_links.append(link)
        #print(sub_links)
        # lấy data từng link
        for sub_link in sub_links:
            print(f"https://www.luxstay.com{sub_link}")
            crawDataOnePage(browser,f"https://www.luxstay.com{sub_link}",time_sleep,file_name)


In [6]:
# crawDataMulPage("https://www.luxstay.com/vi/vietnam/quang-ninh?page=",1,"quangninh_p2.csv",1,15)
# crawDataMulPage("https://www.luxstay.com/vi/vietnam/khanh-hoa/nha-trang?page=",1,"nhatrang_p1.csv",1,10)
# crawDataMulPage("https://www.luxstay.com/vi/vietnam/khanh-hoa/nha-trang?page=",1,"nhatrang_p2.csv",11,20)
# crawDataMulPage("https://www.luxstay.com/vi/vietnam/khanh-hoa/nha-trang?page=",1,"nhatrang_p3.csv",21,30)
# crawDataMulPage("https://www.luxstay.com/vi/vietnam/khanh-hoa/nha-trang?page=",1,"nhatrang_p4.csv",31,34)
# crawDataMulPage("https://www.luxstay.com/vi/vietnam/ba-ria-vung-tau/vung-tau?page=",1,"vungtau_p1.csv",1,10)
# crawDataMulPage("https://www.luxstay.com/vi/vietnam/ba-ria-vung-tau/vung-tau?page=",1,"vungtau_p2.csv",11,20)
# crawDataMulPage("https://www.luxstay.com/vi/vietnam/ba-ria-vung-tau/vung-tau?page=",1,"vungtau_p3.csv",21,31)
# for page in range(1,131,10):
#     crawDataMulPage("https://www.luxstay.com/vi/vietnam/ho-chi-minh?page=",1,f"hcm_p{int(page/10+1)}.csv",page,page+9)
crawDataMulPage("https://www.luxstay.com/vi/vietnam/ho-chi-minh?page=",1,"hcm_p14.csv",31,32)

https://www.luxstay.com/vi/rooms/31440
https://www.luxstay.com/vi/rooms/17234
https://www.luxstay.com/vi/rooms/13388
https://www.luxstay.com/vi/rooms/19874
https://www.luxstay.com/vi/rooms/33408
https://www.luxstay.com/vi/rooms/21848
https://www.luxstay.com/vi/rooms/37180
https://www.luxstay.com/vi/rooms/22971
https://www.luxstay.com/vi/rooms/19798
https://www.luxstay.com/vi/rooms/16647
https://www.luxstay.com/vi/rooms/25954
https://www.luxstay.com/vi/rooms/13380
https://www.luxstay.com/vi/rooms/31722
https://www.luxstay.com/vi/rooms/27006
https://www.luxstay.com/vi/rooms/17080
https://www.luxstay.com/vi/rooms/33817
https://www.luxstay.com/vi/rooms/25736
https://www.luxstay.com/vi/rooms/22618
https://www.luxstay.com/vi/rooms/34653
https://www.luxstay.com/vi/rooms/26126
